In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
from autodm.llm import get_llm
from autodm.roll import Dice, DiceType
from autodm.character import Character
from autodm.locaitons import Location, LocationStore, setup_new_locations
from autodm.storyline import StoryLineWithStoryItems

from rich import print

/Users/michaelfrantz/miniconda3/envs/autodm/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from llama_index.core.program import LLMTextCompletionProgram
from llama_index.vector_stores.lancedb import LanceDBVectorStore
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import PromptTemplate, StorageContext, VectorStoreIndex, Document
from llama_index.core.postprocessor import FixedRecencyPostprocessor
from llama_index.core.node_parser import SimpleNodeParser
from pydantic import BaseModel, Field
from typing import Optional, List, Any, Union
from pathlib import Path
from datetime import datetime
import torch

In [4]:
character = Character.generate(level=0)
print(character)

Character(
    chr_class='wizard',
    chr_race='human',
    name='Aria Moonshadow',
    attributes=Attributes(strength=15, dexterity=1, constitution=8, intelligence=8, wisdom=20, charisma=11),
    level=0,
    max_hp=18,
    hp=18,
    spells=[],
    equipment=[]
)

In [5]:
locations = setup_new_locations()
print(locations)

{
    'The Great Forest of Eldarath': Location(
        type='region',
        name='The Great Forest of Eldarath',
        description='A vast and ancient forest that covers most of the eastern part of the kingdom. It is home to 
many mysterious creatures, magical plants, and ruins from an ancient civilization.',
        parent_name=None,
        parent_description=None
    ),
    "Eldarath's Haven": Location(
        type='city',
        name="Eldarath's Haven",
        description="A bustling city built at the edge of the Great Forest of Eldarath, it serves as a hub for 
traders and adventurers alike who seek to explore the forest's mysteries.",
        parent_name='The Great Forest of Eldarath',
        parent_description='A vast and ancient forest that covers most of the eastern part of the kingdom. It is 
home to many mysterious creatures, magical plants, and ruins from an ancient civilization.'
    )
}

In [6]:
city, region = locations.current, locations.get(locations.current.parent_name)

In [7]:
story = StoryLineWithStoryItems.generate(character, city, region, locations)

In [8]:
print(story)

StoryLineWithStoryItems(
    name='The Lost Ruins of Eldarath',
    details='The party has been tasked with finding and exploring the lost ruins of an ancient civilization 
believed to be located deep within the Great Forest of Eldarath. The ruins are said to hold great power and 
knowledge, but also dangerous secrets that could threaten the entire kingdom if not properly dealt with.',
    completed=False,
    items=StoryItems(
        items=[
            StoryItem(
                name='The Lost Ruins of Eldarath',
                details='The party has been tasked with finding and exploring the lost ruins of an ancient 
civilization believed to be located deep within the Great Forest of Eldarath. The ruins are said to hold great 
power and knowledge, but also dangerous secrets that could threaten the entire kingdom if not properly dealt 
with.',
                completed=False
            ),
            StoryItem(
                name='Encounter with the Ancient Guardian',
                details='The party discovers an enormous stone statue of a long-forgotten deity guarding the 
entrance to the ruins. To progress, they must solve a riddle or defeat the guardian in combat.',
                completed=False
            ),
            StoryItem(
                name='Treasure of Eldarath',
                details='Within the ruins, the party discovers an ancient artifact known as the Treasure of 
Eldarath. The artifact is said to grant immense power to whoever possesses it, but also comes with a great curse.',
                completed=False
            ),
            StoryItem(
                name='The Cursed Curse',
                details='As the party examines the Treasure of Eldarath, they trigger a powerful curse that begins 
to affect their minds and bodies. They must find a way to break the curse before it consumes them entirely.',
                completed=False
            ),
            StoryItem(
                name='The Final Decision',
                details='The party faces a choice: keep the Treasure of Eldarath and risk the consequences, or 
destroy the artifact and save their friends from the curse. The decision they make will determine the fate of the 
kingdom.',
                completed=False
            )
        ]
    )
)

In [3]:
VECTORSTORE_DIR = Path('~').expanduser() / '.autodm/vectorstore'
if not VECTORSTORE_DIR.exists():
    VECTORSTORE_DIR.mkdir(parents=True)

ADVENTURE_LOG_TABLE_NAME = 'adventure_log'

In [4]:
adventure_log_vectorstore = LanceDBVectorStore(uri=str(VECTORSTORE_DIR), table_name=str(ADVENTURE_LOG_TABLE_NAME))
storage_context = StorageContext.from_defaults(vector_store=adventure_log_vectorstore)


In [5]:
def get_device():
    if torch.backends.mps.is_available():
        return 'mps'
    elif torch.cuda.is_available():
        return 'cuda'
    else:
        return 'cpu'

In [15]:
class AdventureLog:
    def __init__(self, lance_uri=VECTORSTORE_DIR, lance_table=ADVENTURE_LOG_TABLE_NAME):
        self.lance_uri = lance_uri
        self.lance_table = lance_table
        self.storage_context = self._setup_storage_context()
        try:
            self.index = self._setup_index()
        except Exception as e:
            self.index = None
            print("Failed to set up index. Will try again on first log.", e)

    def _setup_storage_context(self):
        return StorageContext.from_defaults(
            vector_store=LanceDBVectorStore(
                uri=str(self.lance_uri), table_name=str(self.lance_table)
            )
        )

    def _setup_index(self, doc_or_docs: Union[Document, List[Document]] = None):
        embed_model = HuggingFaceEmbedding(
            "Alibaba-NLP/gte-base-en-v1.5", device=get_device(), trust_remote_code=True
        )
        if doc_or_docs is not None:
            if not isinstance(doc_or_docs, list):
                doc_or_docs = [doc_or_docs]
            return VectorStoreIndex.from_documents(
                doc_or_docs,
                embed_model=embed_model,
                storage_context=self.storage_context,
            )
        else:
            return VectorStoreIndex.from_vector_store(
                vector_store = self.storage_context.vector_store,
                embed_model=embed_model, 
                storage_context=self.storage_context
            )

    def add_entry(self, entry: str, user: str = "dm"):
        doc = Document(
            text=entry, extra_info={"user": user, "timestamp": str(datetime.now())}
        )
        if self.index is None:
            self.index = self._setup_index(doc)
        else:
            self.index.insert(doc)

    def search(self, query: str, top_k: int = 5):
        postprocessor = FixedRecencyPostprocessor(date_key='timestamp')
        query_engine = self.index.as_query_engine(llm=get_llm(output_cls=str), similarity_top_k=top_k, node_postprocessors=[postprocessor])
        return query_engine.query(query)
    
    def clear(self):
        if input("Are you sure you want to clear the adventure log? (y/n): ").lower() == 'y':
            self.storage_context.vector_store.client.drop_table(self.lance_table)
            self.index = None

In [16]:
VectorStoreIndex.from_vector_store

<bound method VectorStoreIndex.from_vector_store of <class 'llama_index.core.indices.vector_store.base.VectorStoreIndex'>>

In [17]:
log = AdventureLog()

/Users/michaelfrantz/miniconda3/envs/autodm/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [14]:
log.add_entry("We look around the city and observe many small buildings surrounding a towering palace in the center of the city.")

/Users/michaelfrantz/miniconda3/envs/autodm/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
[2024-06-02T04:31:58Z WARN  lance::dataset] No existing dataset at /Users/michaelfrantz/.autodm/vectorstore/adventure_log.lance, it will be created


In [15]:
log.add_entry("We enter the tavern in the southeast corner of the city. We see a bar, a few tables, and a stage for performances.")

In [22]:
log.clear()